In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objs as go

import datetime

from pydantic import BaseModel, ConfigDict
from statsmodels.tsa.stattools import adfuller

plt.style.use('ggplot')

In [ ]:
class Position(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    direction: str = ''
    price: float = 0.0
    time: pd.Timestamp = pd.Timestamp.now()

In [ ]:
def process_signals(df, signals):
    df_signal = pd.DataFrame(signals)
    df_result = {}
    for day, df_day in df_signal.groupby(pd.Grouper(level='datetime', freq='D')):
        if df_day.shape[0] == 0:
            continue
        
        if day.day_name() == 'Wednesday':
            entry_long = df_day[df_day.entries == True]
            entry_short = df_day[df_day.short_entries == True]

        if entry_long.shape[0] != 0:
            trade_exit = df_day[df_day.exits == True].index
            trade_entry = entry_long.index
            position = 'long'
        elif entry_short.shape[0] != 0:
            trade_exit = df_day[df_day.short_exits == True].index
            trade_entry = entry_short.index
            position = 'short'
        else:
            continue

        entry_price = df.loc[trade_entry].close.values[0]
        exit_price = df.loc[trade_exit].close.values[0]

        returns = (exit_price - entry_price) / entry_price
        returns = returns * (1 if position == 'long' else -1)

        df_result[trade_exit.date[0]] = {
            'position': position,
            'entry_time': trade_entry.time[0],
            'exit_time': trade_exit.time[0],
            'entry_price': entry_price,
            'exit_price': exit_price,
            'return': returns
        }

    df_result = pd.DataFrame(df_result).T

    return df_result

In [ ]:
with pd.HDFStore('./large_files/data_20240328.h5', mode='r') as store:
    df_ = store.get('data/cl')
df_.info(verbose=3)

In [ ]:
def mean_reversion5(
    df,
    window=30,
    long_entry_threshold=-3.0,
    long_exit_threshold=0, 
    short_entry_threshold=2.0, 
    short_exit_threshold=0,
    fast_sma=5,
    slow_sma=15
):
    long_entry_signals = []
    long_exit_signals = []

    short_entry_signals = []
    short_exit_signals = []
    for day, df_day in df.groupby(pd.Grouper(key='datetime', freq='D')):
        if df_day.shape[0] == 0:
            continue
        
        df_day = df_day.copy()
        df_day['z_score'] = (df_day.close - df_day.close.rolling(window).mean()) / df_day.close.rolling(window).std()

        df_day['sma_slow'] = df_day.close.rolling(fast_sma).mean()
        df_day['sma_fast'] = df_day.close.rolling(slow_sma).mean()
        df_day['down'] = df_day.sma_fast.vbt.crossed_below(df_day.sma_slow).replace(True, 'down').replace(False, np.nan)
        df_day['up'] = df_day.sma_fast.vbt.crossed_above(df_day.sma_slow).replace(True, 'up').replace(False, np.nan)

        df_day['direction'] = df_day.up.combine_first(df_day.down).ffill()

        long_entry = pd.Series(False, index=df_day.index)
        long_exit = pd.Series(False, index=df_day.index)
        short_entry = pd.Series(False, index=df_day.index)
        short_exit = pd.Series(False, index=df_day.index)

        for time, row in df_day.iterrows():
            time = time.time()

            if long_entry.any():
                if ((row.z_score >= long_exit_threshold) and row.direction == 'down') or (time == pd.Timestamp('11:59').time()):
                    long_exit[time] = True
                    break
            elif short_entry.any():
                if ((row.z_score <= short_exit_threshold) and row.direction == 'up') or (time == pd.Timestamp('11:59').time()):
                    short_exit[time] = True
                    break
            elif (time >= pd.Timestamp('10:25').time()) & (time <= pd.Timestamp('10:35').time()) & (row.z_score < long_entry_threshold):
                long_entry[time] = True
            elif (time >= pd.Timestamp('10:25').time()) & (time <= pd.Timestamp('10:35').time()) & (row.z_score > short_entry_threshold):
                short_entry[time] = True

        long_entry_signals.append(long_entry)
        long_exit_signals.append(long_exit)
        short_entry_signals.append(short_entry)
        short_exit_signals.append(short_exit)

    long_entry_signals = pd.concat(long_entry_signals)
    long_exit_signals = pd.concat(long_exit_signals)
    short_entry_signals = pd.concat(short_entry_signals)
    short_exit_signals = pd.concat(short_exit_signals)

    return {
        'entries': long_entry_signals,
        'exits': long_exit_signals,
        'short_entries': short_entry_signals,
        'short_exits': short_exit_signals
    }

In [ ]:
df = df_.set_index('datetime', drop=False)
df = df.loc[df.index.day_of_week.isin([2])]
df = df.loc[df.index.year.isin([2020, 2021, 2022, 2023])]
df = df[(df.index.hour >= 9) & (df.index.hour < 12)]

signals = mean_reversion5(df, fast_sma=5, slow_sma=15)
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    freq='1m',
    allow_partial=True,
    **signals
)
portfolio.stats()

In [ ]:
df_result_mr = process_signals(df, signals)
df_result_mr['position'].value_counts()

In [ ]:
portfolio.plot()

In [ ]:
print((portfolio.annual_returns() * 100).round(4).to_markdown())
weekly_return = (portfolio.total_profit() / (4 * 52)) / 100
cagr = ((1 + weekly_return) ** 52) - 1
print(f'Annualized Returns: {cagr}')

In [ ]:
def mean_reversion5_1(
    df,
    window=30,
    long_entry_threshold=-3.0,
    long_exit_threshold=0, 
    short_entry_threshold=2.0, 
    short_exit_threshold=0,
    fast_sma=5,
    slow_sma=15
):
    long_entry_signals = []
    long_exit_signals = []

    short_entry_signals = []
    short_exit_signals = []
    position = None
    for day, df_day in df.groupby(pd.Grouper(key='datetime', freq='D')):
        if df_day.shape[0] == 0:
            continue
        
        df_day = df_day.copy()
        df_day['z_score'] = (df_day.close - df_day.close.rolling(window).mean()) / df_day.close.rolling(window).std()

        df_day['sma_slow'] = df_day.close.rolling(fast_sma).mean()
        df_day['sma_fast'] = df_day.close.rolling(slow_sma).mean()
        df_day['down'] = df_day.sma_fast.vbt.crossed_below(df_day.sma_slow).replace(True, 'down').replace(False, np.nan)
        df_day['up'] = df_day.sma_fast.vbt.crossed_above(df_day.sma_slow).replace(True, 'up').replace(False, np.nan)

        df_day['direction'] = df_day.up.combine_first(df_day.down).ffill()
        df_day['shift_up'] = (df_day['direction'] == 'up') & (df_day['direction'].shift(1) != 'up')
        df_day['shift_down'] = (df_day['direction'] == 'down') & (df_day['direction'].shift(1) != 'down')

        long_entry = pd.Series(False, index=df_day.index)
        long_exit = pd.Series(False, index=df_day.index)
        short_entry = pd.Series(False, index=df_day.index)
        short_exit = pd.Series(False, index=df_day.index)

        for time, row in df_day.iterrows():
            if position is not None:
                # within 30 minutes
                if (time - position.time) < datetime.timedelta(minutes=60):
                    # long exit
                    if ((position.direction == 'long') and (row.z_score >= long_exit_threshold) and row.shift_down):
                        long_exit[time] = True
                        position = None
                        break
                    # short exit
                    elif ((position.direction == 'short') and (row.z_score <= short_exit_threshold) and row.shift_up):
                        short_exit[time] = True
                        position = None
                        break
                
                # end of Friday
                elif (time.day_name() == 'Friday') and (time.time() == pd.Timestamp('15:59').time()):
                    if position.direction == 'long': long_exit[time] = True
                    elif position.direction == 'short': short_exit[time] = True
                    position = None
                    break

                # after 30 minutes
                else:
                    returns = (row.close - position.price) / position.price
                    returns = returns * (-1 if position.direction == 'short' else 1)

                    # if position.direction == 'long' and row.shift_down and returns > 0.05:
                    #     long_exit[time] = True
                    #     position = None
                    #     break
                    # elif position.direction == 'short' and row.shift_up and returns <= -0.01:
                    #     short_exit[time] = True
                    #     position = None
                    #     break
                    if returns > 0.05 or returns <= -0.005:
                        if position.direction == 'long': long_exit[time] = True
                        elif position.direction == 'short': short_exit[time] = True
                        position = None
                        break

            # entry signals
            elif (time.day_name() == 'Wednesday') & (time.time() >= pd.Timestamp('10:25').time()) & (time.time() <= pd.Timestamp('10:35').time()):
                if (row.z_score < long_entry_threshold):
                    long_entry[time] = True
                    position = Position(direction='long', price=row.close, time=time)

                elif (row.z_score > short_entry_threshold):
                    short_entry[time] = True
                    position = Position(direction='short', price=row.close, time=time)

        long_entry_signals.append(long_entry)
        long_exit_signals.append(long_exit)
        short_entry_signals.append(short_entry)
        short_exit_signals.append(short_exit)

        # if day.day_name() == 'Friday':
        #     assert long_entry.sum() == long_exit.sum(), f'Long signals do no match: {day.date()} {long_entry.sum()} {long_exit.sum()}'
        #     assert short_entry.sum() == short_exit.sum(), f'Short signals do no match: {day.date()} {short_entry.sum()} {short_exit.sum()}' 

    long_entry_signals = pd.concat(long_entry_signals)
    long_exit_signals = pd.concat(long_exit_signals)
    short_entry_signals = pd.concat(short_entry_signals)
    short_exit_signals = pd.concat(short_exit_signals)

    return {
        'entries': long_entry_signals,
        'exits': long_exit_signals,
        'short_entries': short_entry_signals,
        'short_exits': short_exit_signals
    }

In [ ]:
df = df_.set_index('datetime', drop=False)
df = df.loc[df.index.day_of_week.isin([2, 3, 4])]
df = df.loc[df.index.year.isin([2020, 2021, 2022, 2023])]
df = df[(df.index.hour >= 9) & (df.index.hour < 16)]

signals = mean_reversion5_1(df)
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    freq='1m',
    allow_partial=True,
    **signals
)
portfolio.stats()

- 64 / 80 trades were exited within 60 minutes

In [ ]:
portfolio.plot()

In [ ]:
print((portfolio.annual_returns() * 100).round(4).to_markdown())
weekly_return = (portfolio.total_profit() / (4 * 52)) / 100
cagr = ((1 + weekly_return) ** 52) - 1
print(f'Annualized Returns: {cagr}')

In [ ]:
def mean_reversion5_2(
    df,
    window=30,
    long_entry_threshold=-3.0,
    long_exit_threshold=0, 
    short_entry_threshold=2.0, 
    short_exit_threshold=0,
    fast_sma=5,
    slow_sma=15
):
    long_entry_signals = []
    long_exit_signals = []

    short_entry_signals = []
    short_exit_signals = []
    position = None
    for day, df_day in df.groupby(pd.Grouper(key='datetime', freq='D')):
        if df_day.shape[0] == 0:
            continue
        
        df_day = df_day.copy()
        df_day['z_score'] = (df_day.close - df_day.close.rolling(window).mean()) / df_day.close.rolling(window).std()

        df_day['sma_slow'] = df_day.close.rolling(fast_sma).mean()
        df_day['sma_fast'] = df_day.close.rolling(slow_sma).mean()
        df_day['down'] = df_day.sma_fast.vbt.crossed_below(df_day.sma_slow).replace(True, 'down').replace(False, np.nan)
        df_day['up'] = df_day.sma_fast.vbt.crossed_above(df_day.sma_slow).replace(True, 'up').replace(False, np.nan)

        df_day['direction'] = df_day.up.combine_first(df_day.down).ffill()
        df_day['shift_up'] = (df_day['direction'] == 'up') & (df_day['direction'].shift(1) != 'up')
        df_day['shift_down'] = (df_day['direction'] == 'down') & (df_day['direction'].shift(1) != 'down')

        long_entry = pd.Series(False, index=df_day.index)
        long_exit = pd.Series(False, index=df_day.index)
        short_entry = pd.Series(False, index=df_day.index)
        short_exit = pd.Series(False, index=df_day.index)

        for time, row in df_day.iterrows():
            if position is not None:
                # within 30 minutes
                if (time - position.time) < datetime.timedelta(minutes=60):
                    # long exit
                    if ((position.direction == 'long') and (row.z_score >= long_exit_threshold) and row.shift_down):
                        long_exit[time] = True
                        position = None
                        break
                    # short exit
                    elif ((position.direction == 'short') and (row.z_score <= short_exit_threshold) and row.shift_up):
                        short_exit[time] = True
                        position = None
                        break
                
                # end of day
                elif (time.time() == pd.Timestamp('15:59').time()):
                    if position.direction == 'long': long_exit[time] = True
                    elif position.direction == 'short': short_exit[time] = True
                    position = None
                    break

                # after 30 minutes
                else:
                    returns = (row.close - position.price) / position.price
                    returns = returns * (-1 if position.direction == 'short' else 1)
                    if returns > 0.05 or returns <= -0.005:
                        if position.direction == 'long': long_exit[time] = True
                        elif position.direction == 'short': short_exit[time] = True
                        position = None
                        break

            # entry signals
            elif (time.time() >= pd.Timestamp('10:25').time()) & (time.time() <= pd.Timestamp('10:35').time()):
                if (row.z_score < long_entry_threshold):
                    long_entry[time] = True
                    position = Position(direction='long', price=row.close, time=time)

                elif (row.z_score > short_entry_threshold):
                    short_entry[time] = True
                    position = Position(direction='short', price=row.close, time=time)

        long_entry_signals.append(long_entry)
        long_exit_signals.append(long_exit)
        short_entry_signals.append(short_entry)
        short_exit_signals.append(short_exit)

    long_entry_signals = pd.concat(long_entry_signals)
    long_exit_signals = pd.concat(long_exit_signals)
    short_entry_signals = pd.concat(short_entry_signals)
    short_exit_signals = pd.concat(short_exit_signals)

    return {
        'entries': long_entry_signals,
        'exits': long_exit_signals,
        'short_entries': short_entry_signals,
        'short_exits': short_exit_signals
    }

In [ ]:
df = df_.set_index('datetime', drop=False)
df = df.loc[df.index.day_of_week.isin([2])]
df = df.loc[df.index.year.isin([2020, 2021, 2022, 2023])]
df = df[(df.index.hour >= 9) & (df.index.hour < 16)]

signals = mean_reversion5_2(df, fast_sma=5, slow_sma=15)
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    freq='1m',
    allow_partial=True,
    **signals
)
portfolio.stats()

In [ ]:
print((portfolio.annual_returns() * 100).round(4).to_markdown())
print(f"""
    Annualized Returns: {portfolio.annualized_return():.4f}
    Annualized Volatility: {portfolio.annualized_volatility():.4f}
""")

Overall, closing within the day still gives better Sharpe ratio

In [ ]:
# cheat strategy
def cheat_strategy(df):
    window = 30
    long_entry_signals = []
    long_exit_signals = []

    short_entry_signals = []
    short_exit_signals = []
    for day, df_day in df.groupby(pd.Grouper(key='datetime', freq='D')):
        if df_day.shape[0] == 0:
            continue

        long_entry = pd.Series(False, index=df_day.index)
        long_exit = pd.Series(False, index=df_day.index)
        short_entry = pd.Series(False, index=df_day.index)
        short_exit = pd.Series(False, index=df_day.index)
        
        df_day = df_day.copy()

        # get maximum z score
        df_day['z_score'] = (df_day.close - df_day.close.rolling(window).mean()) / df_day.close.rolling(window).std()

        df_10 = df_day[(df_day.datetime.dt.time >= pd.Timestamp('10:25').time()) &( df_day.datetime.dt.time <= pd.Timestamp('10:35').time())]
        max_idx = df_10.z_score.abs().idxmax()
        z_score = df_day.loc[max_idx, 'z_score']

        is_entry = False
        # entry signal
        if z_score >= 2:
            short_entry[max_idx] = True
            is_entry = True
        elif z_score <= -3:
            long_entry[max_idx] = True
            is_entry = True

        # exit signal
        if is_entry:
            df_later = df_day[df_day.datetime.dt.time > max_idx.time()]
            if df_day.loc[max_idx, 'z_score'] > 0:
                exit_point = df_later.close.idxmin()
                short_exit[exit_point] = True
            else:
                exit_point = df_later.close.idxmax()
                long_exit[exit_point] = True

        assert long_entry.sum() == long_exit.sum(), f'Long signals do no match: {day}'
        assert short_entry.sum() == short_exit.sum(), f'Short signals do no match: {day}'

        long_entry_signals.append(long_entry)
        long_exit_signals.append(long_exit)
        short_entry_signals.append(short_entry)
        short_exit_signals.append(short_exit)
        
    long_entry_signals = pd.concat(long_entry_signals)
    long_exit_signals = pd.concat(long_exit_signals)
    short_entry_signals = pd.concat(short_entry_signals)
    short_exit_signals = pd.concat(short_exit_signals)

    return {
        'entries': long_entry_signals,
        'exits': long_exit_signals,
        'short_entries': short_entry_signals,
        'short_exits': short_exit_signals
    }

In [ ]:
df_cheat = df_.set_index('datetime', drop=False)
df_cheat = df_cheat.loc[df_cheat.index.day_of_week.isin([2])]
df_cheat = df_cheat.loc[df_cheat.index.year.isin([2020, 2021, 2022, 2023])]
df_cheat = df_cheat[(df_cheat.index.hour >= 9) & (df_cheat.index.hour < 12)]

signals = cheat_strategy(df_cheat)
portfolio = vbt.Portfolio.from_signals(
    close=df_cheat['close'],
    freq='1m',
    allow_partial=True,
    **signals
)
portfolio.stats()

In [ ]:
df_result_cheat = process_signals(df, signals)
df_result_cheat['position'].value_counts()

In [ ]:
portfolio.plot()

In [ ]:
print((portfolio.annual_returns() * 100).round(4).to_markdown())
weekly_return = (portfolio.total_profit() / (4 * 52)) / 100
cagr = ((1 + weekly_return) ** 52) - 1
print(f'Annualized Returns: {cagr}')